In [7]:
import os
os.environ['OMP_NUM_THREADS'] = '1'

import sys
sys.path.append(os.path.join(os.getcwd(), *tuple(['..'])))
import argparse

from typing import Callable, Dict, List, Optional, Set
from collections import OrderedDict
import pdb
import numpy as np
import torch
from model_mrcnn import _default_mrcnn_config, build_default
from features import build_features
from features import transforms as T
from utils.engine import evaluate
import torchvision
from utils.helper_functions import evaluate_metrics, get_outputs, compute_iou, evaluate_mask_rcnn
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support

In [8]:
test_config = dict(
    batch_size = 2,
    num_classes=4,
    device_id =0
)

In [9]:
#model_name = os.path.join("/gladstone/finkbeiner/steve//work/data/npsad_data/vivek/models/stellar-pyramid-31_mrcnn_model_100.pth")
#model_name = os.path.join("/gladstone/finkbeiner/steve//work/data/npsad_data/vivek/models/lyric-puddle-40_mrcnn_model_80.pth")
#model_name = os.path.join("/gladstone/finkbeiner/steve//work/data/npsad_data/vivek/models/visionary-salad-42_mrcnn_model_80.pth")
#model_name = os.path.join("/gladstone/finkbeiner/steve//work/data/npsad_data/vivek/models/devout-oath-43_mrcnn_model_80.pth")
model_name = os.path.join("/gladstone/finkbeiner/steve//work/data/npsad_data/vivek/models/treasured-flower-64_mrcnn_model_10.pth")

In [10]:
dataset_test_location = '/gladstone/finkbeiner/steve//work/data/npsad_data/vivek/Datasets/amyb_wsi/val_v2'
test_patient_ids = os.listdir(dataset_test_location)
test_patient_ids.remove('.DS_Store')
collate_fn = lambda _: tuple(zip(*_)) # one-liner, no need to import

In [11]:
test_patient_ids

['XE19-037_1_AmyB_1',
 'XE18-003_1_AmyB_1',
 'XE11-039_1_AmyB_1',
 'XE07-049_1_AmyB_1',
 'XE18-066_1_AmyB_1']

In [12]:
model_config = _default_mrcnn_config(num_classes=1+test_config["num_classes"]).config
model = build_default(model_config, im_size=1024)
model.load_state_dict(torch.load(model_name))
device = torch.device('cpu')
device = torch.device('cuda', test_config['device_id'])
model = model.to(device) 
model.eval()

all_patients_val_metrics = pd.DataFrame()
for t in range(len(test_patient_ids)):
    if len(os.listdir(os.path.join(dataset_test_location,test_patient_ids[t],"images")))==0:
        continue
    test_ds = build_features.AmyBDataset(os.path.join(dataset_test_location,test_patient_ids[t]),T.Compose([T.ToTensor()]))
    test_loader = torch.utils.data.DataLoader(test_ds, batch_size=test_config['batch_size'], shuffle=False, num_workers=4, collate_fn=collate_fn)

    final_metric_df =pd.DataFrame()
    for threshold in [0.25,0.5,0.75,0.9,0.95]:
        f1_mean_list = []
        labels_matched_list = []
        actual_label_list = []
        pred_label_list = []
        for i, (images, targets) in enumerate(test_loader):
            images = [image.to(device) for image in images]
            targets = [dict([(k, v.to(device)) for k, v in target.items()]) for target in targets]
            outputs = model.forward(images, targets)
            masks, labels = get_outputs(outputs, threshold)
            f1_mean, labels_matched, actual_labels,pred_labels =  evaluate_metrics(targets, masks, labels)
            if len(f1_mean)>0 or len(labels_matched)>0:
                #print(" Validation f1 mean score:", f1_mean, " perc labels matched", labels_matched)
                f1_mean_list.extend(f1_mean)
                labels_matched_list.extend(labels_matched)
                actual_label_list.extend(actual_labels)
                pred_label_list.extend(pred_labels)
        metric_df = pd.DataFrame({"f1_score":f1_mean_list,"labels_matched":labels_matched_list,"actual_labels":actual_label_list,
                                  "pred_labels":pred_label_list})
        metric_df["threshold"] = threshold
        if  len(final_metric_df)>0:
            final_metric_df=pd.concat([final_metric_df,metric_df],axis=0, ignore_index=True)
        else:
            final_metric_df=metric_df
    final_metric_df["patient_id"]=test_patient_ids[t]
    if len(all_patients_val_metrics)>0:
        all_patients_val_metrics=pd.concat([final_metric_df,all_patients_val_metrics],axis=0, ignore_index=True)
    else:
        all_patients_val_metrics=final_metric_df


/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'backbone_name' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB (GPU 0; 15.71 GiB total capacity; 13.60 GiB already allocated; 33.75 MiB free; 14.01 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 

In [15]:
pd.concat([final_metric_df,metric_df],axis=0, ignore_index=True)

,f1_score,labels_matched,actual_labels,pred_labels,threshold
0,NaN,0,1,2,0.95
1,NaN,0,1,2,0.95
2,NaN,0,2,2,0.95
3,NaN,0,1,2,0.95
4,0.648812,1,2,2,0.95
5,NaN,0,1,2,0.95
6,NaN,0,2,2,0.95
7,NaN,0,1,2,0.95
8,NaN,0,1,2,0.95
9,NaN,0,2,2,0.95
